# Example to get energies (total, electronic, nuclear, heat of formation, orbital), charges, dipole, forces, density matrix.

In [2]:
import torch
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.ElectronicStructure import Electronic_Structure
torch.set_default_dtype(torch.float64)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
#device = torch.device('cpu')

In [3]:
%%time

### create molecule object:
species = torch.as_tensor([[8,6,1,1],
                           [8,6,1,1],
                           [8,8,6,0]], # zero-padding for batching
                          dtype=torch.int64, device=device)

coordinates = torch.tensor([
                              [
                               [0.00,    0.00,    0.00],
                               [1.22,    0.00,    0.00],
                               [1.82,    0.94,    0.00],
                               [1.82,   -0.94,    0.00]
                              ],
                              [
                               [0.00,    0.00,    0.00],
                               [1.22,    0.00,    0.00],
                               [1.82,    0.94,    0.00],
                               [1.82,   -0.94,    0.00]
                              ],
                              [
                               [0.00,    0.00,    0.00],
                               [1.23,    0.00,    0.00],
                               [1.82,    0.94,    0.00],
                               [0.0,0.0,0.0]            # zero-padding for batching
                              ]
                            ], device=device)

const = Constants().to(device)

seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM3, PM6, PM6_SP. PM6_SP is PM6 without d-orbitals. Effectively, PM6 for the first two rows of periodic table
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [0,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True, # store orbital energies
                   }

molecules = Molecule(const, seqm_parameters, coordinates, species).to(device)

### Create electronic structure driver:
esdriver = Electronic_Structure(seqm_parameters).to(device)

### Run esdriver on molecules:
esdriver(molecules)

scf direct step  :  32 | E[   0]: -864.99403762 | MAX ΔE[   0]:  -0.00000000 | MAX ΔDM[   2]:    0.0000017 | MAX ΔDM_ij[   0]:  0.0000091  | N not converged: 0
CPU times: user 70.3 ms, sys: 26 ms, total: 96.4 ms
Wall time: 89.9 ms


In [4]:
molecules.nocc

tensor([6, 6, 8])

In [5]:
print(' Total Energy (eV):\n', molecules.Etot)
print('\n Electronic Energy (eV): ', molecules.Eelec)
print('\n Nuclear Energy (eV):\n', molecules.Enuc)
print('\n Heat of Formation (ev):\n', molecules.Hf)
print('\n HOMO-LUMO gap (eV):\n', molecules.e_gap)
print('\n Orbital energies (eV):\n', molecules.e_mo)
print('\n Charges:\n', molecules.q)
print('\n Dipole:\n', molecules.d)
print('\n Forces (eV/A):\n', molecules.force)

 Total Energy (eV):
 tensor([-475.5819, -475.5819, -759.2461])

 Electronic Energy (eV):  tensor([ -864.9940,  -864.9940, -1504.4581])

 Nuclear Energy (eV):
 tensor([389.4122, 389.4122, 745.2120])

 Heat of Formation (ev):
 tensor([-1.3620, -1.3620,  6.3444])

 HOMO-LUMO gap (eV):
 tensor([11.5926, 11.5926,  9.3511])

 Orbital energies (eV):
 tensor([[-39.2320, -25.0148, -17.1717, -16.2774, -14.6391, -10.7560,   0.8365,
           3.1526,   3.9792,   6.0948,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000],
        [-39.2320, -25.0148, -17.1717, -16.2774, -14.6391, -10.7560,   0.8365,
           3.1526,   3.9792,   6.0948,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000],
        [-46.6531, -35.1909, -24.5378, -20.3546, -20.2776, -15.5338, -11.1725,
         -11.1534,  -1.8023,  -0.0903,   2.2116,   6.4507,   0.0000,   0.0000,
           0.0000,   0.0000]])

 Charges:
 tensor([[-0.2727,  0.1406,  0.0661,  0.0661],
        [-0.2727,  0.1406,  0.0

In [6]:
print('\n Density matrix:\n', molecules.dm)


 Density matrix:
 tensor([[[ 1.9076e+00, -2.7516e-01,  2.2050e-16, -1.3169e-16,  1.3504e-01,
          -2.8679e-01,  7.7822e-16, -3.7068e-17,  6.1923e-03,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  6.1923e-03,  0.0000e+00,  0.0000e+00,
           0.0000e+00],
         [-2.7516e-01,  1.1584e+00, -1.8598e-16,  4.2720e-17,  5.0980e-01,
          -7.8705e-01, -7.7202e-17, -2.7444e-16, -9.9626e-02,  0.0000e+00,
           0.0000e+00,  0.0000e+00, -9.9626e-02,  0.0000e+00,  0.0000e+00,
           0.0000e+00],
         [ 2.2050e-16, -1.8598e-16,  1.9075e+00, -4.3339e-16,  5.2737e-16,
           6.3222e-16,  3.0871e-01,  9.5402e-16, -2.0135e-01,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  2.0135e-01,  0.0000e+00,  0.0000e+00,
           0.0000e+00],
         [-1.3169e-16,  4.2720e-17, -4.3339e-16,  1.2991e+00,  2.2016e-17,
          -2.7600e-17,  9.8450e-17,  9.5421e-01,  4.1693e-16,  0.0000e+00,
           0.0000e+00,  0.0000e+00, -5.9944e-16,  0.0000e+00,  0.0000e+00,
         

In [7]:
molecules.coordinates

Parameter containing:
tensor([[[ 0.0000,  0.0000,  0.0000],
         [ 1.2200,  0.0000,  0.0000],
         [ 1.8200,  0.9400,  0.0000],
         [ 1.8200, -0.9400,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000],
         [ 1.2200,  0.0000,  0.0000],
         [ 1.8200,  0.9400,  0.0000],
         [ 1.8200, -0.9400,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000],
         [ 1.2300,  0.0000,  0.0000],
         [ 1.8200,  0.9400,  0.0000],
         [ 0.0000,  0.0000,  0.0000]]], requires_grad=True)